<h1>Table of Contents &lt;br&gt;&lt;/br&gt;<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Data-loading" data-toc-modified-id="Imports-and-Data-loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Data loading</a></span></li></ul></div>

## Imports and Data loading

In [13]:
# Display plots inline
%matplotlib inline

# Autoreload all package before excecuting a call
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [16]:
import evsim
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


ENV_NAME = 'evsim-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Set the prediction accuracy of simulation
env.prediction_accuracy(10)

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.

dqn.fit(env, nb_steps=6700, visualize=False, verbose=1, log_interval=100)

# After training is done, we save the final weights.

dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.

dqn.test(env, nb_episodes=1, visualize=False)


/home/morty/uni/evsim/env/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Could not seed environment <FleetEnv<evsim-v0>>
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 1)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 16)                32        
_________________________________________________________________
activation_25 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_26 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_27 (Activation)   (None, 16)                0         
__________

/home/morty/uni/evsim/env/lib/python3.7/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


100/100 [==============================] - 4s 35ms/step - reward: 0.6395
Interval 2 (100 steps performed)
100/100 [==============================] - 2s 20ms/step - reward: -35.8053
Interval 3 (200 steps performed)
100/100 [==============================] - 2s 19ms/step - reward: 0.0000e+00
Interval 4 (300 steps performed)
100/100 [==============================] - 2s 19ms/step - reward: -44.7366
Interval 5 (400 steps performed)
100/100 [==============================] - 2s 20ms/step - reward: 0.5886
Interval 6 (500 steps performed)
100/100 [==============================] - 2s 20ms/step - reward: -134.8743
Interval 7 (600 steps performed)
100/100 [==============================] - 2s 21ms/step - reward: -163.0912
Interval 8 (700 steps performed)
100/100 [==============================] - 2s 20ms/step - reward: -107.5341
Interval 9 (800 steps performed)
100/100 [==============================] - 2s 20ms/step - reward: 0.7136
Interval 10 (900 steps performed)
100/100 [===================